In [1]:
cd /home/grad/Desktop/pietro/denovo/2/xx

/home/grad/Desktop/pietro/denovo/2/xx


In [2]:
# ====================================================================
# [ 1. CONFIGURAZIONE ]
# ====================================================================

class Config:
    # Percorsi file
    SMILES_FILE = "/home/grad/Desktop/pietro/denovo/2/compacted_smiles.txt"
    SMARTS_RX_FILE = "/home/grad/Desktop/pietro/denovo/2/SMART_RX/smartsrx-main/smartsrx.json"
    TRAIN_DATA_PATH = "/home/grad/Desktop/pietro/denovo/2/train_data.pkl"
    VAL_DATA_PATH = "/home/grad/Desktop/pietro/denovo/2/val_data.pkl"
    VOCAB_PATH = "/home/grad/Desktop/pietro/denovo/2/vocab.json"
    # Iperparametri Modello
    EMBED_DIM = 512
    TRANSFORMER_LAYERS = 6
    TRANSFORMER_HEADS = 6
    FF_DIM = 2048
    DROPOUT_RATE = 0.10
    L2_REG = 1e-4
    
    # Training
    BATCH_SIZE = 48
    EPOCHS = 200
    MAX_LENGTH = 140         # Aumentato leggermente per ospitare 3 parti
    GRADIENT_CLIP = 1.0
    VALID_RATIO = 0.1
    
    # Curriculum & Augmentation
    CURRICULUM_START_COMPLEXITY = 10
    CURRICULUM_COMPLEXITY_STEP = 5
    LOSS_STABILITY_THRESHOLD = 0.01
    WARMUP_EPOCHS = 5
    AUGMENT_PROB = 0.1
    
    # Generazione
    TEMPERATURE = 1.0
    GEN_NUM = 10
    PRINT_EVERY = 100
    STEPS_PER_EPOCH = 1000

config = Config()

In [3]:
import os
import sys
import time
import random
import re
import logging
import json
import pickle
import numpy as np
import tensorflow as tf
from threading import Lock
from typing import List, Tuple, Optional

# RDKit Imports
from rdkit import Chem
from rdkit.Chem import MolFromSmiles, MolToSmiles
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.Chem import FilterCatalog  # Necessario per SMARTS-RX
from sklearn.model_selection import train_test_split

# TensorFlow Imports
from tensorflow.keras.layers import Layer, Embedding, Input, LayerNormalization, MultiHeadAttention, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback, ModelCheckpoint



# Configurazione Logger
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

# ====================================================================
# [ 2. GESTIONE SMARTS-RX & SCAFFOLD ]
# ====================================================================

# Variabili Globali Catalogo
SMARTS_CATALOG = None

def initialize_smarts_catalog():
    """Carica il catalogo SMARTS-RX."""
    global SMARTS_CATALOG
    if SMARTS_CATALOG is not None: return

    try:
        with open(config.SMARTS_RX_FILE, "rt", encoding="utf-8") as f:
            data = json.load(f)
    except Exception as e:
        logger.error(f"Errore caricamento SMARTS JSON: {e}")
        sys.exit(1)

    catalog = FilterCatalog.FilterCatalog()
    count = 0
    for entry in data.get("data", []):
        name = entry.get("specific_type")
        smarts = entry.get("smarts")
        if name and smarts:
            pattern = Chem.MolFromSmarts(smarts)
            if pattern:
                catalog.AddEntry(FilterCatalog.FilterCatalogEntry(name, FilterCatalog.SmartsMatcher(pattern)))
                count += 1
    SMARTS_CATALOG = catalog
    logger.info(f"Catalogo SMARTS-RX caricato: {count} regole.")

def get_smarts_fingerprint(mol: Chem.Mol) -> List[str]:
    """Restituisce lista nomi gruppi funzionali presenti."""
    if not mol or SMARTS_CATALOG is None: return []
    matches = SMARTS_CATALOG.GetMatches(mol)
    # Set per unicità, sorted per ordine deterministico
    return sorted(list(set([m.GetDescription() for m in matches])))

def get_murcko_scaffold_tokens(mol: Chem.Mol) -> List[str]:
    """Estrae scaffold e lo tokenizza."""
    if not mol: return []
    try:
        scaffold = MurckoScaffold.GetScaffoldForMol(mol)
        smi = Chem.MolToSmiles(scaffold, canonical=True, isomericSmiles=False)
        if not smi: return []
        return robust_tokenize(smi)
    except:
        return []

# ====================================================================
# [ 3. HELPER CHIMICI BASE ]
# ====================================================================

def robust_tokenize(smiles: str) -> list:
    pattern = (
        r"(\[[^\[\]]{1,6}\]|"                 # atomi in parentesi quadre
        r"Br|Cl|Si|Na|Mg|Mn|Ca|Fe|Zn|Se|Li|K|Al|B|"  # elementi multi-char
        r"R[0-9]|r[0-9]|a[0-9]|"             # ring labels
        r"[A-Za-z0-9@+\-\\\/\(\)=#\$\.\%,])"  # singoli caratteri, incluso '%'
    )
    tokens = re.findall(pattern, smiles)
    stack = []
    for t in tokens:
        if t.startswith('['): stack.append(t)
        if t.endswith(']'):
            if not stack: return []
            stack.pop()
    return tokens

def validate_and_fix_smiles(smiles: str) -> str:
    try:
        mol = Chem.MolFromSmiles(smiles, sanitize=True)
        if mol is None: return None
        try: Chem.Kekulize(mol, clearAromaticFlags=True)
        except: pass
        return Chem.MolToSmiles(mol, canonical=True, isomericSmiles=False)
    except: return None

def randomize_smiles(smiles: str, num_versions: int = 1) -> List[str]:
    mol = Chem.MolFromSmiles(smiles)
    if not mol: return []
    res = []
    try:
        s = Chem.MolToSmiles(mol, doRandom=True, canonical=False)
        if s: res.append(s)
    except: pass
    return res

def compute_complexity_from_tokens(tokens: List[str]) -> int:
    smiles = ''.join(tokens)
    try:
        mol = Chem.MolFromSmiles(smiles)
        if not mol: return 999
        return Chem.GetSSSR(mol) + smiles.count('(')
    except: return 999

# ====================================================================
# [ 4. PREPROCESSING DATASET (IBRIDO) ]
# ====================================================================

def process_dataset(data: List[str]) -> Tuple[List[Tuple[List[str], List[str], List[str]]], List[str], int]:
    """
    Output Tuple: (SMARTS_TOKENS, SCAFFOLD_TOKENS, TARGET_TOKENS)
    """
    initialize_smarts_catalog()
    
    processed = []
    all_tokens = set()
    # Abbiamo 2 separatori ora: SMARTS|SEP|SCAFFOLD|SEP|TARGET
    # Token speciali totali nella sequenza: START, SEP, SEP, END = 4 token logici (ma SEP è lo stesso ID)
    
    logger.info("Inizio Preprocessing Ibrido (SMARTS + Scaffold)...")
    
    for idx, s in enumerate(data):
        if idx % 10000 == 0: logger.info(f"Processati {idx}/{len(data)}")
            
        fixed = validate_and_fix_smiles(s)
        if not fixed: continue
        
        mol = Chem.MolFromSmiles(fixed)
        if not mol: continue

        # 1. Target (SMILES)
        tokens_target = robust_tokenize(fixed)
        if not tokens_target: continue
        
        # 2. Scaffold (Murcko)
        tokens_scaffold = get_murcko_scaffold_tokens(mol)
        
        # 3. SMARTS-RX
        tokens_smarts = get_smarts_fingerprint(mol)
        
        # Calcolo Lunghezza: START + SMARTS + SEP + SCAFFOLD + SEP + TARGET + END
        total_len = 1 + len(tokens_smarts) + 1 + len(tokens_scaffold) + 1 + len(tokens_target) + 1

        if 5 <= total_len <= config.MAX_LENGTH:
            processed.append((tokens_smarts, tokens_scaffold, tokens_target))
            all_tokens.update(tokens_target)
            all_tokens.update(tokens_scaffold)
            all_tokens.update(tokens_smarts) # Aggiunge nomi come "Alcohol", "Ketone"

    # Costruzione Vocabolario
    vocab = ['<PAD>', '<START>', '<END>', '<SEP>', '<UNK>']
    vocab.extend(sorted(list(all_tokens)))
    
    # Ricalcolo Max Len
    lengths = []
    for t_smart, t_scaf, t_targ in processed:
        lengths.append(1 + len(t_smart) + 1 + len(t_scaf) + 1 + len(t_targ) + 1)
        
    if processed:
        max_len = min(int(np.percentile(lengths, 99)), config.MAX_LENGTH)
    else:
        max_len = config.MAX_LENGTH
    
    return processed, vocab, max_len

# ====================================================================
# [ 5. CLASSI THREAD-SAFE ]
# ====================================================================

class ThreadSafeIterator:
    def __init__(self, iterator):
        self.iterator = iterator
        self.lock = Lock()
    def __iter__(self): return self
    def __next__(self):
        with self.lock: return next(self.iterator)

def threadsafe_generator(func):
    def wrapper(*args, **kwargs): return ThreadSafeIterator(func(*args, **kwargs))
    return wrapper

# ====================================================================
# [ 6. GENERATORE CURRICULUM (3 PARTI) ]
# ====================================================================

class CurriculumSmilesGenerator:
    def __init__(self, processed_data, vocab: List[str]):
        self.char2idx = {c: i for i, c in enumerate(vocab)}
        self.idx2char = {i: c for c, i in self.char2idx.items()}
        self.original_data = []
        
        # data = (smarts, scaffold, target)
        for smarts, scaffold, target in processed_data:
            comp = compute_complexity_from_tokens(target)
            self.original_data.append(((smarts, scaffold, target), comp))
            
        valid_comps = [c for _, c in self.original_data if c != 999]
        self.max_complexity = max(valid_comps) if valid_comps else 0
        self.current_complexity = config.CURRICULUM_START_COMPLEXITY
        self.available_data = self._filter_data()
        # Per novelty check
        self.train_smiles = {''.join(t) for (_, _, t), _ in self.original_data}
        self.lock = Lock()
    
    def _filter_data(self):
        filtered = [dp for dp, c in self.original_data if c <= self.current_complexity]
        return filtered if filtered else [dp for dp, _ in self.original_data]
    
    def update_complexity(self, epoch: int, loss_diff: float = None):
        with self.lock:
            if loss_diff is not None and loss_diff < config.LOSS_STABILITY_THRESHOLD:
                self.current_complexity = min(self.current_complexity + config.CURRICULUM_COMPLEXITY_STEP, self.max_complexity)
            else:
                if epoch <= config.WARMUP_EPOCHS:
                    incr = int((self.max_complexity - config.CURRICULUM_START_COMPLEXITY) * (epoch / config.WARMUP_EPOCHS))
                    self.current_complexity = config.CURRICULUM_START_COMPLEXITY + incr
                else:
                    self.current_complexity = self.max_complexity
            
            self.available_data = self._filter_data()
            if not self.available_data:
                self.available_data = [dp for dp, _ in self.original_data]
                logger.warning("Reset available_data (fallback).")
    
    @threadsafe_generator
    def __call__(self):
        PAD_IDX = self.char2idx['<PAD>']
        UNK_IDX = self.char2idx.get('<UNK>', PAD_IDX)

        while True:
            inputs = np.full((config.BATCH_SIZE, config.MAX_LENGTH), PAD_IDX, dtype=np.int32)
            targets = np.full_like(inputs, PAD_IDX)
            
            for i in range(config.BATCH_SIZE):
                with self.lock:
                    try:
                        data_pair = random.choice(self.available_data)
                    except:
                        self.available_data = [dp for dp, _ in self.original_data]
                        data_pair = random.choice(self.available_data)
                
                # Unpacking 3 parti
                t_smarts, t_scaf, t_targ = data_pair

                # Augmentation solo sul target
                curr_target = t_targ
                if random.random() < config.AUGMENT_PROB:
                    try:
                        aug = randomize_smiles(''.join(t_targ), 1)
                        if aug:
                            tok = robust_tokenize(aug[0])
                            if tok: curr_target = tok
                    except: pass

                # STRUTTURA: [START] SMARTS [SEP] SCAFFOLD [SEP] TARGET [END]
                seq = (['<START>'] + 
                       t_smarts + ['<SEP>'] + 
                       t_scaf + ['<SEP>'] + 
                       curr_target + ['<END>'])
                
                padded_seq = (seq + ['<PAD>'] * config.MAX_LENGTH)[:config.MAX_LENGTH]
                indices = [self.char2idx.get(t, UNK_IDX) for t in padded_seq]
                
                inputs[i] = indices
                targets[i, :-1] = inputs[i][1:]
                targets[i, -1] = PAD_IDX
                
            yield inputs, targets

    def get_dataset(self):
        return tf.data.Dataset.from_generator(
            self.__call__,
            output_signature=(
                tf.TensorSpec(shape=(config.BATCH_SIZE, config.MAX_LENGTH), dtype=tf.int32),
                tf.TensorSpec(shape=(config.BATCH_SIZE, config.MAX_LENGTH), dtype=tf.int32)
            )
        ).prefetch(tf.data.AUTOTUNE)

# ====================================================================
# [ 7. MODELLO TRANSFORMER ]
# ====================================================================

# ====================================================================
# [ 7. ARCHITETTURA MODELLO & LOSS ]
# ====================================================================

# 1. DEFINIZIONE LOSS GLOBALE (Necessario per il salvataggio/caricamento)
def smoothed_loss(y_true, y_pred):
    """
    Calcola la Cross Entropy ignorando i token di padding (0).
    """
    # Cast a int per le labels
    y_true_int = tf.cast(y_true, tf.int32)
    # Maschera: 1 dove c'è un token reale, 0 dove c'è padding
    mask = tf.cast(tf.math.not_equal(y_true, 0), tf.float32)
    
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true_int, logits=y_pred)
    
    # Media pesata (somma loss valida / numero token validi)
    return tf.reduce_sum(loss * mask) / (tf.reduce_sum(mask) + 1e-9)

# 2. LAYERS CUSTOM
class DynamicPositionalEncoding(Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        
    def build(self, input_shape):
        # Usiamo la lunghezza massima definita nella config per pre-calcolare la matrice
        max_len = config.MAX_LENGTH
        pos = np.arange(max_len)[:, np.newaxis]
        i = np.arange(self.embed_dim)[np.newaxis, :]
        
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(self.embed_dim))
        angle_rads = pos * angle_rates
        
        angle_rads[:, 0::2] = tf.math.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = tf.math.cos(angle_rads[:, 1::2])
        
        self.pos_encoding = tf.cast(angle_rads[np.newaxis, ...], dtype=tf.float32)
        
    def call(self, inputs):
        # Taglia la matrice posizionale alla lunghezza attuale del batch
        seq_len = tf.shape(inputs)[1]
        return inputs + self.pos_encoding[:, :seq_len, :]
        
    def get_config(self):
        return {**super().get_config(), "embed_dim": self.embed_dim}

class ImprovedTransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ffn_dim, rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ffn_dim = ffn_dim
        self.rate = rate
        
        self.mha = MultiHeadAttention(
            num_heads=num_heads, 
            key_dim=embed_dim, 
            kernel_regularizer=tf.keras.regularizers.l2(config.L2_REG)
        )
        self.ffn = tf.keras.Sequential([
            Dense(ffn_dim, activation="gelu", kernel_regularizer=tf.keras.regularizers.l2(config.L2_REG)),
            Dense(embed_dim, kernel_regularizer=tf.keras.regularizers.l2(config.L2_REG))
        ])
        self.ln1 = LayerNormalization(epsilon=1e-6)
        self.ln2 = LayerNormalization(epsilon=1e-6)
        self.d1 = Dropout(rate)
        self.d2 = Dropout(rate)

    def call(self, inputs, training=False, mask=None):
        # Creazione Maschera Causale (impedisce di guardare il futuro)
        seq_len = tf.shape(inputs)[1]
        causal_mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
        
        attn_output = self.mha(inputs, inputs, attention_mask=causal_mask)
        out1 = self.ln1(inputs + self.d1(attn_output, training=training))
        
        ffn_output = self.ffn(out1)
        return self.ln2(out1 + self.d2(ffn_output, training=training))
        
    def get_config(self):
        return {**super().get_config(), "embed_dim": self.embed_dim, "num_heads": self.num_heads, "ffn_dim": self.ffn_dim, "rate": self.rate}

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, embed_dim, warmup_steps=10000):
        super().__init__()
        # FIX: Usa tf.cast per evitare errori di tipo
        self.embed_dim = tf.cast(embed_dim, tf.float32)
        self.warmup_steps = tf.cast(warmup_steps, tf.float32)

    def __call__(self, step):
        step = tf.cast(step, tf.float32) + 1e-9
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.embed_dim) * tf.math.minimum(arg1, arg2)

    def get_config(self):
        return {"embed_dim": float(self.embed_dim), "warmup_steps": float(self.warmup_steps)}

# 3. COSTRUZIONE MODELLO
def build_improved_model(vocab_size: int) -> Model:
    inputs = Input(shape=(config.MAX_LENGTH,))
    
    # FIX: mask_zero=False perché gestiamo tutto manualmente (causal mask + loss)
    x = Embedding(vocab_size, config.EMBED_DIM, mask_zero=False)(inputs)
    
    x = DynamicPositionalEncoding(config.EMBED_DIM)(x)
    x = Dropout(config.DROPOUT_RATE)(x)
    
    for _ in range(config.TRANSFORMER_LAYERS):
        x = ImprovedTransformerBlock(
            config.EMBED_DIM, 
            config.TRANSFORMER_HEADS, 
            config.FF_DIM, 
            rate=config.DROPOUT_RATE
        )(x)
        
    outputs = Dense(vocab_size)(x)
    
    # Setup Optimizer & Compile
    opt = tf.keras.optimizers.Adam(
        learning_rate=CustomSchedule(config.EMBED_DIM), 
        clipnorm=config.GRADIENT_CLIP
    )
    
    model = Model(inputs, outputs)
    # Usiamo la loss globale definita sopra
    model.compile(optimizer=opt, loss=smoothed_loss)
    
    return model

# ====================================================================
# [ 8. MONITORAGGIO (Parsing Complesso) ]
# ====================================================================

class CustomTensorBoard(tf.keras.callbacks.TensorBoard):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}; lr = self.model.optimizer.learning_rate
        logs['lr'] = lr(epoch).numpy() if isinstance(lr, tf.keras.optimizers.schedules.LearningRateSchedule) else lr.numpy()
        super().on_epoch_end(epoch, logs)

class EnhancedTrainingMonitor(Callback):
    def __init__(self, val_gen: CurriculumSmilesGenerator):
        super().__init__()
        self.val_gen = val_gen
        self.best_val_loss = np.inf; self.prev_val_loss = None

    def generate_sample(self, num: int):
        generated, valid = [], []
        PAD, START, END, SEP = [self.val_gen.char2idx[k] for k in ['<PAD>','<START>','<END>','<SEP>']]
        input_seq = np.full((1, config.MAX_LENGTH), PAD, dtype=np.int32)
        
        for _ in range(num):
            input_seq.fill(PAD); input_seq[0, 0] = START
            for t in range(1, config.MAX_LENGTH):
                logits = self.model(input_seq, training=False)[0, t-1]
                probs = tf.nn.softmax(logits / config.TEMPERATURE).numpy()
                if np.sum(probs) < 1e-6: break
                sampled = np.random.choice(len(probs), p=probs)
                input_seq[0, t] = sampled
                if sampled == END: break
            
            indices = input_seq[0].tolist()
            raw_tokens = [self.val_gen.idx2char[i] for i in indices if i not in {PAD, START}]
            
            # Parsing Logic: START [SMARTS] SEP [SCAFFOLD] SEP [TARGET] END
            # Dobbiamo trovare l'ultimo SEP per ottenere il target
            smi_str = ""
            try:
                # Conta quanti SEP ci sono
                sep_indices = [i for i, x in enumerate(raw_tokens) if x == '<SEP>']
                if len(sep_indices) >= 2:
                    # Prendi tutto dopo il SECONDO SEP
                    target_tokens = raw_tokens[sep_indices[1]+1:]
                    smi_str = "".join(target_tokens).split('<END>')[0]
                elif len(sep_indices) == 1:
                    # Fallback: forse ha saltato uno step
                    target_tokens = raw_tokens[sep_indices[0]+1:]
                    smi_str = "".join(target_tokens).split('<END>')[0]
                else:
                    smi_str = "".join(raw_tokens).split('<END>')[0]
            except: pass
            
            final = validate_and_fix_smiles(smi_str)
            if final: valid.append(final)
            generated.append(final or smi_str)
            
        return generated, valid

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        if self.prev_val_loss and 'val_loss' in logs:
            diff = (self.prev_val_loss - logs['val_loss']) / self.prev_val_loss
            self.val_gen.update_complexity(epoch, diff)
        if logs.get('val_loss', np.inf) < self.best_val_loss: self.best_val_loss = logs['val_loss']
        self.prev_val_loss = logs.get('val_loss')
        
        if (epoch + 1) % config.PRINT_EVERY == 0:
            gen, val = self.generate_sample(config.GEN_NUM)
            validity = len(val) / config.GEN_NUM
            novel = len([s for s in val if s not in self.val_gen.train_smiles])
            logger.info(f"\nEPOCH {epoch+1}: Validity {validity:.1%} | Novelty {novel}/{len(val)}")
            if val: logger.info(f"Sample: {val[0]}")


2026-02-16 11:13:55.956180: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-16 11:13:55.970245: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771236835.987294 1229876 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771236835.992639 1229876 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771236836.005599 1229876 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [4]:
# ====================================================================
# [ 9. MAIN EXECUTION ]
# ====================================================================

if __name__ == "__main__":
    logger.info("🚀 START TRAINING: MULTI-CONDITIONAL (SMARTS + SCAFFOLD -> TARGET)")
    
    # 1. Verifica Files
    if not os.path.exists(config.SMILES_FILE):
        logger.error("File SMILES non trovato."); sys.exit(1)
    if not os.path.exists(config.SMARTS_RX_FILE):
        logger.error("File SMARTS JSON non trovato."); sys.exit(1)

    with open(config.SMILES_FILE) as f:
        raw_smiles = [line.strip() for line in f if line.strip()]
    logger.info(f"Caricati {len(raw_smiles)} SMILES.")

    # 2. Preprocessing
    processed, vocab, max_len = process_dataset(raw_smiles)
    config.MAX_LENGTH = max_len
    logger.info(f"Vocab: {len(vocab)} | MaxLen: {max_len}")

    # 3. Salvataggi
    with open("vocab.json", "w") as f: json.dump(vocab, f)
    with open("char2idx.pkl", "wb") as f: pickle.dump({c:i for i,c in enumerate(vocab)}, f)
    with open("idx2char.pkl", "wb") as f: pickle.dump({i:c for i,c in enumerate(vocab)}, f)

    # 4. Split (stratificato su target)
    stratify = [min(len(t[2]), 20) for t in processed]
    try:
        train_data, val_data = train_test_split(processed, test_size=config.VALID_RATIO, stratify=stratify, random_state=42)
    except:
        train_data, val_data = train_test_split(processed, test_size=config.VALID_RATIO, random_state=42)

    # 5. Esempio di Log
    if train_data:
        s_smart, s_scaf, s_targ = train_data[0]
        logger.info(f"\n--- ESEMPIO SEQ ---")
        logger.info(f"SMARTS:   {s_smart}")
        logger.info(f"SCAFFOLD: {''.join(s_scaf)}")
        logger.info(f"TARGET:   {''.join(s_targ)}")
        logger.info("-------------------")

    # 6. Avvio
    train_gen = CurriculumSmilesGenerator(train_data, vocab)
    val_gen = CurriculumSmilesGenerator(val_data, vocab)
    
    model = build_improved_model(len(vocab))
    model.summary()
    
    callbacks = [
        CustomTensorBoard(log_dir=f"logs/hybrid_{int(time.time())}"),
        EnhancedTrainingMonitor(val_gen),
        ModelCheckpoint("best_hybrid_model.keras", monitor="val_loss", save_best_only=True)
    ]

    try:
        model.fit(
            train_gen.get_dataset(),
            epochs=config.EPOCHS,
            steps_per_epoch=config.STEPS_PER_EPOCH,
            validation_data=val_gen.get_dataset(),
            validation_steps=max(1, len(val_data)//config.BATCH_SIZE),
            callbacks=callbacks
        )
        model.save("final_hybrid_model.keras")
        logger.info("Training Completato.")
    except KeyboardInterrupt:
        logger.warning("Training interrotto.")
        model.save("interrupted_hybrid.keras")

2026-02-16 11:14:07,078 [INFO] 🚀 START TRAINING: MULTI-CONDITIONAL (SMARTS + SCAFFOLD -> TARGET)
2026-02-16 11:14:07,324 [INFO] Caricati 891004 SMILES.
2026-02-16 11:14:07,409 [INFO] Catalogo SMARTS-RX caricato: 406 regole.
2026-02-16 11:14:07,409 [INFO] Inizio Preprocessing Ibrido (SMARTS + Scaffold)...
2026-02-16 11:14:07,410 [INFO] Processati 0/891004
[11:14:07] SMILES Parse Error: syntax error while parsing: SMILES
[11:14:07] SMILES Parse Error: Failed parsing SMILES 'SMILES' for input: 'SMILES'

KeyboardInterrupt



In [2]:
pwd

'/home/grad/Desktop/pietro/denovo/2'

✓ train_data salvato: 801902 molecole


In [ ]:
if __name__ == "__main__":
    logger.info("🚀 AVVIO SCRIPT: CARICAMENTO DIRETTO E ADDESTRAMENTO")
    
    # 1. CONTROLLO FILE ESISTENTI
    if not (os.path.exists(config.TRAIN_DATA_PATH) and 
            os.path.exists(config.VAL_DATA_PATH) and 
            os.path.exists(config.VOCAB_PATH)):
        logger.error(f"❌ File di cache mancanti! Esegui prima lo script di salvataggio.")
        logger.error(f"   Cercati: {config.TRAIN_DATA_PATH}, {config.VAL_DATA_PATH}, {config.VOCAB_PATH}")
        sys.exit(1)

    # 2. CARICAMENTO DATI
    logger.info("📂 Caricamento dati da disco (Pickle)...")
    try:
        with open(config.TRAIN_DATA_PATH, "rb") as f: train_data = pickle.load(f)
        with open(config.VAL_DATA_PATH, "rb") as f: val_data = pickle.load(f)
        with open(config.VOCAB_PATH, "r") as f: vocab = json.load(f)
        logger.info(f"✅ Dati caricati: {len(train_data)} Train, {len(val_data)} Val, {len(vocab)} Vocab")
    except Exception as e:
        logger.error(f"❌ Errore durante il caricamento: {e}")
        sys.exit(1)

    # 3. SETUP GENERATORI
    logger.info("⚙️ Inizializzazione Generatori...")
    train_gen = CurriculumSmilesGenerator(train_data, vocab)
    val_gen = CurriculumSmilesGenerator(val_data, vocab)

    # 4. SETUP MODELLO
    logger.info("🏗️ Costruzione Modello...")
    model = build_improved_model(len(vocab))
    model.summary()

    # 5. AVVIO TRAINING
    logger.info("🔥 Inizio Training Loop...")
    
    callbacks = [
        CustomTensorBoard(log_dir=f"logs/run_loaded_{int(time.time())}"),
        EnhancedTrainingMonitor(val_gen),
        ModelCheckpoint("best_hybrid_model.keras", monitor="val_loss", save_best_only=True)
    ]
    try:
        model.fit(
            train_gen.get_dataset(),
            epochs=config.EPOCHS,
            steps_per_epoch=config.STEPS_PER_EPOCH,
            validation_data=val_gen.get_dataset(),
            validation_steps=max(1, len(val_data)//config.BATCH_SIZE),
            callbacks=callbacks
        )
        model.save("final_hybrid_model.keras")
        logger.info("🏆 Training Completato.")
    except KeyboardInterrupt:
        logger.warning("⚠️ Training interrotto dall'utente.")
        model.save("interrupted_hybrid.keras")

2026-01-17 08:16:25,181 [INFO] 🚀 AVVIO SCRIPT: CARICAMENTO DIRETTO E ADDESTRAMENTO
2026-01-17 08:16:25,182 [INFO] 📂 Caricamento dati da disco (Pickle)...
2026-01-17 08:16:32,331 [INFO] ✅ Dati caricati: 801902 Train, 89101 Val, 370 Vocab
2026-01-17 08:16:32,332 [INFO] ⚙️ Inizializzazione Generatori...
2026-01-17 08:18:46,356 [INFO] 🏗️ Costruzione Modello...
I0000 00:00:1768634327.314430 3641570 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8888 MB memory:  -> device: 0, name: NVIDIA RTX A2000 12GB, pci bus id: 0000:65:00.0, compute capability: 8.6


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 140)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 140, 512)       │       189,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dynamic_positional_encoding     │ (None, 140, 512)       │             0 │
│ (DynamicPositionalEncoding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 140, 512)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ improved_transformer_block      │ (None, 140, 512)       │    10,502,144 │
│ (ImprovedTransformerBlock)      │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ improved_transformer_block_1    │ (None, 140, 512)       │    10,502,144 │
│ (ImprovedTransformerBlock)      │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ improved_transformer_block_2    │ (None, 140, 512)       │    10,502,144 │
│ (ImprovedTransformerBlock)      │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ improved_transformer_block_3    │ (None, 140, 512)       │    10,502,144 │
│ (ImprovedTransformerBlock)      │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ improved_transformer_block_4    │ (None, 140, 512)       │    10,502,144 │
│ (ImprovedTransformerBlock)      │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ improved_transformer_block_5    │ (None, 140, 512)       │    10,502,144 │
│ (ImprovedTransformerBlock)      │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 140, 370)       │       189,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 63,392,114 (241.82 MB)

 Trainable params: 63,392,114 (241.82 MB)

 Non-trainable params: 0 (0.00 B)

2026-01-17 08:18:49,267 [INFO] 🔥 Inizio Training Loop...


Epoch 1/200


I0000 00:00:1768634342.773462 3646777 service.cc:152] XLA service 0x72cd40002f80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1768634342.773488 3646777 service.cc:160]   StreamExecutor device (0): NVIDIA RTX A2000 12GB, Compute Capability 8.6
2026-01-17 08:19:04.083809: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1768634347.564042 3646777 cuda_dnn.cc:529] Loaded cuDNN version 90300
2026-01-17 08:19:11.107499: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_153', 20 bytes spill stores, 20 bytes spill loads

2026-01-17 08:19:11.241825: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_153', 29

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 745s 689ms/step - loss: 4.2979 - val_loss: 2.8281 - lr: 4.4194e-05
Epoch 2/200
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 693s 693ms/step - loss: 2.7320 - val_loss: 2.2598 - lr: 8.8388e-05
Epoch 3/200
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 693s 693ms/step - loss: 2.1763 - val_loss: 1.7930 - lr: 1.3258e-04
Epoch 4/200
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 705s 705ms/step - loss: 1.7228 - val_loss: 1.3908 - lr: 1.7678e-04
Epoch 5/200
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 694s 695ms/step - loss: 1.3580 - val_loss: 1.1254 - lr: 2.2097e-04
Epoch 6/200
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 695s 695ms/step - loss: 1.0903 - val_loss: 0.9329 - lr: 2.6517e-04
Epoch 7/200
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 696s 696ms/step - loss: 0.9280 - val_loss: 0.8389 - lr: 3.0936e-04
Epoch 8/200
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 706s 707ms/step - loss: 0.8465 - val_loss: 0.7779 - lr: 3.5355e-04
Epoch 9/200
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 692s 693ms/step - loss: 0.8019 - val_loss: 0.7653 - lr: 3.9775e-04
Epoch 10/200
1000/1000

TypeError: <class 'keras.src.models.functional.Functional'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'keras.src.models.functional', 'class_name': 'Functional', 'config': {}, 'registered_name': 'Functional', 'build_config': {'input_shape': None}, 'compile_config': None}.

Exception encountered: Could not locate class 'ImprovedTransformerBlock'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': None, 'class_name': 'ImprovedTransformerBlock', 'config': {'name': 'improved_transformer_block', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None, 'shared_object_id': 139862845535040}, 'embed_dim': 512, 'num_heads': 6, 'ffn_dim': 2048, 'rate': 0.1}, 'regi